In [22]:
import numpy as np
from IPython.display import display, Math

from utils.libration_sense import (
    get_xf, du2km, initial_state_parser, get_maxdev_sampling_ellipsoid, km2du, kmS2vu, vu2kms, get_maxdev_linear_ellipsoid
)
from utils.formula_creators import (
   alpha_finder_of_n, n_finder
)

In [23]:
ORBIT_TYPE = "L2"
ORBIT_NUMBER = 21
xf = get_xf(ORBIT_TYPE, ORBIT_NUMBER)

GRID_DENSITY = 11
AMOUNT_OF_POINTS = 10_000

SEED = False
REUSE_NOISE = True

In [24]:
n, A_normed, y_du = n_finder(
            ORBIT_TYPE,
            ORBIT_NUMBER,
            xf,
            grid_density=GRID_DENSITY,
            seed=SEED,
            reuse_noise=REUSE_NOISE,
            amount_of_points=AMOUNT_OF_POINTS,
        )

alpha, deviation_max_du = alpha_finder_of_n(A_normed, y_du, n)
deviation_max_km = du2km(deviation_max_du)

In [25]:
print('Степень n = ', n, sep='')

Степень n = 2.0000007448681996


In [26]:
alpha, deviation_max = alpha_finder_of_n(A_normed, y_du, n)

def deviation_analytic(n, alpha, std_pos_normed, std_vel_normed):
    return np.power(alpha[0]*np.power(std_pos_normed, n) + alpha[1]*np.power(std_vel_normed, n), 1/n)

formula = f"d = \\sqrt[{np.around(n, 4)}]{{{np.around(alpha[0], 2)}\\cdot \\sigma_{{pos}}^{{{np.around(n, 4)}}} + {np.around(alpha[1], 2)}\\cdot \\sigma_{{vel}}^{{{np.around(n, 4)}}}}}\\cdot{np.around(du2km(deviation_max), 3)} \\ [km]"

std_pos, std_vel = km2du(10.), kmS2vu(0.09e-3)
std_pos_max, std_vel_max = km2du(17), kmS2vu(0.11e-3)

np.random.seed(42)
print(f"Пробная точка: [sigma_pos = {np.around(du2km(std_pos), 1)} км, sigma_vel = {vu2kms(std_vel)*1000} м/с]")
print(f"Крайние значения данных: [sigma_pos_max = {3} км, sigma_vel_max = {0.03} м/с]")
# print('Точное значение: ', du2km(get_maxdev_sampling_ellipsoid(ORBIT_TYPE, ORBIT_NUMBER, xf, std_pos, std_vel, amount_of_points=11000)))
print('Точное значение: ', du2km(get_maxdev_linear_ellipsoid( xf, std_pos, std_vel, radius=3.0)))
print('Значение, вычисленное по аппроксимирующей формуле: ', deviation_analytic(n, alpha, std_pos/std_pos_max, std_vel/std_vel_max) * du2km(deviation_max))
display(Math(formula))

Пробная точка: [sigma_pos = 10.0 км, sigma_vel = 0.09000000000000001 м/с]
Крайние значения данных: [sigma_pos_max = 3 км, sigma_vel_max = 0.03 м/с]
Точное значение:  33047.116153009265
Значение, вычисленное по аппроксимирующей формуле:  33047.11618537373


<IPython.core.display.Math object>